# Accessing data with CliMetLab

Here are a few examples on how to get data into python using CliMetLab.

More examples can be found in the CliMetLab documentation (https://climetlab.readthedocs.io/en/latest/examples.html).

## How to run this exercise

This exercise is in the form of a [Jupyter notebook](https://jupyter.org/). It can be "run" in a number of free cloud based environments (see two options below). These require no installation. When you click on one of the links below ([`Open in Colab`](https://colab.research.google.com/github/ecmwf-projects/mooc-machine-learning-weather-climate/blob/main/tier_2/data_handling/01-accessing-data.ipynb) or [`Launch in Deepnote`](https://deepnote.com/launch?url=https://github.com/ecmwf-projects/mooc-machine-learning-weather-climate/blob/main/tier_2/data_handling/01-accessing-data.ipynb)) you will be prompted to create a free account, after which you will see the same page you see here. You can run each block of code by selecting shift+control repeatedly, or by selecting the "play" icon. 

Advanced users may wish to run this exercise on their own computers by first installing [Python](https://www.python.org/downloads/), [Jupyter](https://jupyter.org/install) and [CliMetLab](https://climetlab.readthedocs.io/en/latest/installing.html).

<style>
td, th {
   border: 1px solid white;
   border-collapse: collapse;
}
</style>
<table align="left">
  <tr>
    <th>Run the tutorial via free cloud platforms: </th>
    <th><a href="https://colab.research.google.com/github/ecmwf-projects/mooc-machine-learning-weather-climate/blob/main/tier_2/data_handling/01-accessing-data.ipynb">
        <img src = "https://colab.research.google.com/assets/colab-badge.svg" alt = "Colab"></th>
    <th><a href="https://deepnote.com/launch?url=https://github.com/ecmwf-projects/mooc-machine-learning-weather-climate/blob/main/tier_2/data_handling/01-accessing-data.ipynb">
        <img src = "https://deepnote.com/buttons/launch-in-deepnote-small.svg" alt = "Kaggle"></th>
  </tr>
</table>

## Loading various sources

Now let's begin exploring data with CliMetLab. We will start by loading various "sources". A source is a "something providing data". It needs some parameters to define the actual location of the data. For instance, a source can be a URL, a remote or local server, a database, or a file, etc.

Below are a few examples of using ``cml.load_source()``.

Notice how: 
- The API to retrieve the data is similar.
- Credentials are requested if needed with appropriate help message, then saved locally.
- Data is provided as pandas.Dataframe or xarray.Dataset.

Let's download some files for the examples:

In [ ]:
!test -f test.grib || wget https://github.com/ecmwf/climetlab/raw/main/docs/examples/test.grib
!test -f test.nc || wget https://github.com/ecmwf/climetlab/raw/main/docs/examples/test.nc
!test -f temp.bufr || wget http://download.ecmwf.int/test-data/metview/gallery/temp.bufr

## Install and import CliMetLab

In [ ]:
!pip install climetlab

In [ ]:
import climetlab as cml

## Read data

How to read data from a GRIB file:

In [ ]:
ds = cml.load_source('file', 'test.grib')
for field in ds:
	cml.plot_map(field)
ds.to_xarray()

How to read data from a netCDF file:

In [ ]:
ds = cml.load_source('file', 'test.nc')
for field in ds:
	cml.plot_map(field)
ds.to_xarray()

How to read data from a BUFR file:

In [ ]:
ds = cml.load_source('file', 'temp.bufr')
df = ds.to_pandas(
    columns=(
        "stationNumber",
        "latitude",
        "longitude",
        "data_datetime",
        "pressure",
        "airTemperature",
    ),
    filters={},
)
cml.plot_map(df, projection="global")
df

The purpose of the "file" source is to open **any** file, CliMetLab guesses what type of file is provided.

## Reading from a URL `cml.load_source("url", URL)`

How to read data from a GRIB URL (using the climetlab source "url")

In [ ]:
ds = cml.load_source('url', 'https://github.com/ecmwf/climetlab/raw/main/docs/examples/test.grib')
for field in ds:
	cml.plot_map(field)
ds.to_xarray()

In [ ]:
cml.load_source('url', 'https://github.com/ecmwf/climetlab/raw/main/docs/examples/test.nc')
cml.load_source('url', 'http://download.ecmwf.int/test-data/metview/gallery/temp.bufr')

In [ ]:
# Downloading 32M of data.
URL = "https://www.ncei.noaa.gov/data/international-best-track-archive-for-climate-stewardship-ibtracs/v04r00/access/csv/ibtracs.SP.list.v04r00.csv"
data = cml.load_source("url", URL)
pd = data.to_pandas()

In [ ]:
uma = pd[pd.NAME == "UMA:VELI"]
cml.plot_map(uma, style="cyclone-track")

As shown below, data can be read from different sources, the API is similar: "load_source" with keywords to specify the data requested.

In [ ]:
source = cml.load_source(
    "cds",
    "reanalysis-era5-single-levels",
    variable=["2t", "msl"],
    product_type="reanalysis",
    area=[50, -50, 20, 50],
    date="2011-12-02", # ["2011-12-02", "2011-12-03"] for several dates
    time="12:00",
)
for s in source:
    cml.plot_map(s)

In [ ]:
source = cml.load_source(
    "mars",
    param=["2t", "msl"],
    levtype="sfc",
    area=[50, -50, 20, 50],
    grid=[1, 1],
    date="2011-12-02",
    
)
for s in source:
    cml.plot_map(s)
source.to_xarray()

In [ ]:
source = cml.load_source(
    "ecmwf-open-data",
    param=["2t", "msl"],
)
for s in source:
    cml.plot_map(s, title=True)
source.to_xarray()

In [ ]:
source = cml.load_source(
    "mars",
    type="ofb",
    obsgroup="conv",
    time="12",
    format="odb",
    reportype=16001,
    date="2020-05-18",
)
pd = source.to_pandas()

## Dataset

A dataset is a clearly identified set of data. It has a name, can be cited, has documentation and, hopefully, somebody is maintaining it.

A dataset is an object created using cml.load_dataset(name, arg1, arg2=..., ...)

- The name is a string that uniquely identifies the dataset.
- The argument(s) arg1 and keyword argument(s) arg2 can be used to specify a subset of the dataset.
- The data can be accessed using methods such as to_xarray() or to_pandas() or other.
- Relevant metadata are attached directly to the dataset to provide additional information such as a URL, a citation, licence, etc.

### Example dataset: hurricane database

In [ ]:
atlantic = cml.load_dataset("hurricane-database", bassin="atlantic")
df = atlantic.to_pandas()

### Example dataset: part of ERA5 

In [ ]:
import climetlab as cml
ds = cml.load_dataset("era5-temperature", 
#    period=(1979, 1999), # This can be long to run.
    period=(1979, 1981),
    domain="France",
    time=12,
 )

In [ ]:
ds.to_xarray()

Once the data has been downloaded. It is cached locally. The following will not download any additional data.


In [ ]:
import climetlab as cml
ds = cml.load_dataset("era5-temperature", 
    period=1980,
    domain="France",
    time=12,
 )

If the dataset is not found, CliMetLab provides the list of available datasets:

In [ ]:
import climetlab as cml
cml.load_dataset('hurican-database-with-typo') # Fails


Additional datasets are defined with python plugins (via pip install), more details on this process in the "Dataset plugins" notebook.

More  examples can be found at (https://climetlab.readthedocs.io/en/latest/examples.html).

## Exercises
Find the mean temperature (2m temperature) in Italy in 1980, at 6am.


**Solutions:**

In [13]:
import climetlab as cml
data = cml.load_dataset("era5-temperature", 
    period=1980,
    domain="Italy",
    time=6,
)
ds = data.to_xarray()
float(ds.mean().t2m)

2022-12-13 11:08:05,737 INFO Welcome to the CDS
2022-12-13 11:08:05,738 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2022-12-13 11:08:05,793 INFO Request is queued
2022-12-13 11:08:06,831 INFO Request is running
2022-12-13 11:08:38,267 INFO Request is completed
2022-12-13 11:08:38,268 INFO Downloading https://download-0017.copernicus-climate.eu/cache-compute-0017/cache/data1/adaptor.mars.internal-1670929685.99353-9453-18-be2b5584-5d61-461e-bca6-aa550874bc41.grib to /data/mafp/tmp-climetlab/cds-retriever-aa6308824a9555852822234e1accd9c9e28a360d7682a543c22c8dafa837e8e6.cache.tmp (2.8M)
2022-12-13 11:08:38,655 INFO Download rate 7.4M/s   


284.1121520996094